
# Build a basic ME model

We will try to build an ME model from the NC_000913.2 Genbank file, the iJO1366 M model, and the complex reconstruction from iJL1650-ME

In [ ]:
# python imports
import re
import json
from os.path import join
import cPickle

# third party imports
import pandas
import escher
import cobra.test
import cloudpickle

# ecoli me
import ecolime
from ecolime.flat_files import *
from ecolime.ecoli_k12 import *
from ecolime import ribosome, tRNA_charging, transcription

from minime.util import dogma
from minime import *
from minime.util import building
from minime.solve.algorithms import binary_search, fva, solve_at_growth_rate
from minime.solve.symbolic import compile_expressions

In [ ]:
import ecolime

In [ ]:
ijo = cobra.test.create_test_model('ecoli')

# Colton TODOs 
- [x] 1) look into all of the free modifications in the original ME and make sure none of thes reactions are blocked
- [x] 2) Frameshifts (b2891)
- [x] 3) selenocysteine
- [x] 4) Non AUG start codons (15%)
- [x] 5) moaD is a modifier protein that never has to be made
- [x] 6) Correct rpL7/12_mod_1:acetyl modification
    - in iOL as a free modification. could be corrected
- 7) Remaining Genes: 
     - pflC, citC, norW - formed but not used in iJL1678
     - FusA, tsf - translation reactions need audited and improved
     - [x] paaD - FeS transfer chaperone. This process needs audited and improved
     - [x] pnp, orn, rhlB - degradosome
     - secB, tatE - Translocation Pathways
     - lgt, lspA - part of translocation. Not sure how to incorporate-ask Joanne about this
- 8) look into lgt (b2828) should this be added to the model?
- 9) confirm whether the palmitate modification for lpp is necessary
- 10) apply ndh1,ndh2 1:1 flux split

In [ ]:
from minime.util.mass import compute_RNA_mass

## Begin by loading metabolites and build Metabolic reactions

In [ ]:
me = MEmodel('iJO1366-ME')
m_model = ecolime.get_m_model()
# some of the "metabolites" in iJO1366 "M" model are actually complexes. We pass those in
# so they get created as complexes, not metabolites.
complex_list = []
complex_list.extend(i.id for i in m_model.metabolites if i.id.startswith("CPLX"))
complex_list.extend(i.id for i in m_model.metabolites if i.id.startswith("EG"))
complex_list.extend(i.id for i in m_model.metabolites if "-MONOMER" in i.id)
complex_list.extend(i.id for i in m_model.metabolites if "-CPLX" in i.id)
complex_list.extend(i.id for i in m_model.metabolites if "_mod_" in i.id)
# temp fix
complex_list.extend(i.id for i in m_model.metabolites if i.id.startswith("Isc"))
complex_list.extend(i.id for i in m_model.metabolites if i.id.startswith("Suf"))
complex_list = set(complex_list)
building.add_m_model_content(me, m_model, complex_metabolite_ids=complex_list)

# if the bounds of this metabolite aren't open, model uses wrong reactions
me.reactions.EX_pqq_e.lower_bound = -1000


In [ ]:
me.global_info["translation_terminators"] = ribosome.translation_stop_dict
me.global_info["met_start_codons"] = {"AUG", "GUG", "UUG", "AUU", "CUG"}
# "Translational capacity" of ogranism
me.global_info['kt'] =  4.5 #(in h-1)scott 2010, RNA-to-protein curve fit
me.global_info['r0'] =  0.087 #scott 2010, RNA-to-protein curve fit
me.global_info['k_deg'] = 1.0/5. * 60.0  # 1/5 1/min 60 min/h # h-1

# Molecular mass of RNA component of ribosome
me.global_info['m_rr'] = 1700. # in kDa

# Average molecular mass of an amino acid
me.global_info['m_aa'] = 109. / 1000. # in kDa

# Proportion of RNA that is rRNA
me.global_info['f_rRNA'] = .86
me.global_info['m_nt'] = 324. / 1000. # in kDa
me.global_info['f_mRNA'] = .02

# tRNA associated global information
me.global_info['m_tRNA'] = 25000. / 1000. # in kDA
me.global_info['f_tRNA'] = .12

me.global_info['RNA_polymerase'] = {'CPLX0-221', 'RNAPE-CPLX', 'CPLX0-222', 'RNAP32-CPLX',
                                    'RNAP54-CPLX', 'RNAP70-CPLX', 'RNAPS-CPLX'}
#current values used in model (double check these)
#me.translation_info['c_ribo'] = 1700./0.109/0.86/3600. #mrr/(maa*f_rRNA)

In [ ]:
for met in me.global_info['RNA_polymerase']:
    RNAP_obj = RNAP(met)
    me.add_metabolites(RNAP_obj)

In [ ]:
df = pandas.read_table('../ecolime/modification.txt', names=['mod', 'formula','na'])
df = df.drop('na', axis=1).set_index('mod').dropna(how='any')
me.global_info['modification_formulas'] = df.T.to_dict()

In [ ]:
essential_list = [
    #'trdox_c',
#    'fldox_c',
    'LI_c',
    'cs_e',
    'tl_c', # added in old ME but not metioned in supplement tables
#    'fldrd_c',
#    'trdrd_c',
    'Oxidized_c',
#    'btn_c',
#    '3fe4s_c',
    'palmitate_c',
    'C10H8O5_c', 'C9H9O4_c', # for tRNA modifications
    'NiFeCoCN2_c', 'acetyl_c',
    'RNase_m5','RNase_m16','RNase_m23'] # RNAses are gaps in model, fix how cs is added to

for met_id in essential_list:
    r = cobra.Reaction("EX_" + met_id)
    me.add_reaction(r)
    r.reaction = met_id + " <=> "

In [ ]:
# add oxidative damage reaction to form 3fe4s 
fes_damage = cobra.Reaction('4fe4s_oxidation')
me.add_reaction(fes_damage)
fes_damage.reaction = '4fe4s_c -> fe2_c + 3fe4s_c'

### Add tRNA mods (iOL uses keffs of 64 for all modifications)

In [ ]:
for mod, components in get_tRNA_modification_procedures().items():
    tRNA_mod = ModificationData(mod, me)
    tRNA_mod.enzyme = components['machines']
    tRNA_mod.stoichiometry = components['metabolites']
    tRNA_mod.keff = 65. # iOL uses 65 for all tRNA mods
    if 'carriers' in components.keys():
        for carrier, stoich in components['carriers'].items():
            if stoich < 0:
                tRNA_mod.enzyme += [carrier]
            tRNA_mod.stoichiometry[carrier] = stoich

## DNA replication

In [ ]:
data = ComplexData('DNA_polymerase', me)
stoichiometry = {
    "b3702": 1,  # dnaA (initiator)
    "b4052": 6,  # dnaB (helicase)
    "b4361": 1,  # dnaC
    "b3066": 3,  # dnaG (primase)
    # CPLXO-3803 (dna polymerase iii, holoenzyme)
    # core
    "b0184": 3,  # dnaE
    "b0215": 3,  # dnaQ
    "b1842": 3,  # holE
    # holo
    "b0470": 5,  # dnaX
    "b1099": 1,  # holB
    "b0640": 1,  # holA
    "b3701": 4, # dnaN
    "b4259": 4, # holC
    "b4372": 4, # holD
    
    # CPLX0-2425 (dna gyrase)
    "b2231": 2,  # gyrA
    "b3699": 2,  # gyrB
}
data.stoichiometry = {"protein_" + bnum: value for bnum, value in stoichiometry.items()}
data.create_complex_formation()

DNA_replication = SummaryVariable("DNA_replication")
me.add_reaction(DNA_replication)
DNA_replication.add_metabolites({
    "datp_c": -int(0.02616 * 1000),
    "dctp_c": -int(0.02701 * 1000),
    "dgtp_c": -int(0.02701 * 1000),
    "dttp_c": -int(0.02616 * 1000),
    "ppi_c": int(0.106 * 1000),
})
DNA_biomass = Constraint("DNA_biomass")
DNA_biomass.elements = {e: abs(v) for e, v in DNA_replication.check_mass_balance().items()}
DNA_replication.add_metabolites({DNA_biomass: 1})
DNA_replication.add_metabolites({'DNA_polymerase': -1e-4})

## Build ribosome and RNA Polymerase

### Add translation-related Subreactions

In [ ]:
for subreaction in ribosome.translation_subreactions:
    subreaction_data = SubreactionData(subreaction, me)
    enzymes = ribosome.translation_subreactions[subreaction]['enzyme']
    subreaction_data.enzyme = enzymes
    

for subreaction in transcription.transcription_subreactions:
    subreaction_data = SubreactionData(subreaction, me)
    enzymes = transcription.transcription_subreactions[subreaction]['enzymes']
    subreaction_data.stoichiometry = transcription.transcription_subreactions[subreaction]['stoich']
    subreaction_data.enzyme = enzymes


In [ ]:
data = ComplexData('RNA_degradosome', me)
data.stoichiometry =  {'Eno_dim_mod_4:mg2':1,
                      'Pnp_trim': 1,
                      'RNase_E_tetra_mod_2:zn2': 1,
                      'RhlB_dim': 1,
                      'Orn_dim_mod_2:mg2':1}
data.create_complex_formation()

Selenocysteine and start codons

In [ ]:
# TODO contrant type is 'trna efficiency' does this change anything?
# 1 machine + 1 atp + 1 aa + 1 h2o --> 1 machine-amp + 1 h + 1 ppi
# 1 machine-amp + 1 free tRNA --> 1 machine + 1 amp + 1 charged tRNA
subreaction_data = SubreactionData('fmet_addition_at_START', me)
subreaction_data.enzyme = ['InfB_mono',  # Start codon loader enzyme and formylmethyltransferase
                           'Fmt_mono_mod_mg2_mod_k']

# iOL had h_c:1 for fmet addtion but this is not mass balanced
subreaction_data.stoichiometry = {'10fthf_c': -1, 'thf_c':1, #'h_c': 1, 
                                 'generic_tRNA_START_met__L_c': -1, 'met__L_c': -1}

subreaction_data = SubreactionData('sec_addition_at_UGA', me)
subreaction_data.enzyme = ['SelA_deca_mod_10:pydx5p',
                           'SelB_mono'] # Selenocysteine loader enzyme
subreaction_data.stoichiometry = {'h_c': 1, 'h2o_c': -1, 'selnp_c':-1, 'pi_c':1,
                                 'generic_tRNA_UGA_cys__L_c': -1, 'cys__L_c': -1}

# create subreaction for each codon
for codon in dogma.codon_table:
    if dogma.codon_table[codon] == '*':
        stop_codon = codon.replace('T','U')
        stop_enzyme = ribosome.translation_stop_dict.get(stop_codon)
        me.add_metabolites([Complex(stop_enzyme)])
        
        subreaction_data = SubreactionData(stop_codon + '_' + stop_enzyme +
                                           '_mediated_termination', me)
        subreaction_data.enzyme = stop_enzyme
        subreaction_data.stoichiometry = {}
    else:
        full_aa = dogma.amino_acids[dogma.codon_table[codon]]
        amino_acid = full_aa.split('_')[0]
        subreaction_data = SubreactionData(amino_acid + '_addition_at_' \
                                           + codon.replace('T','U'), me)
        tRNA = 'generic_tRNA_' + codon.replace('T','U') + '_' + full_aa
        subreaction_data.enzyme = 'generic_Tuf' # Default AA loader enzyme
        subreaction_data.stoichiometry = {full_aa: -1, 'atp_c': -1,# 'h2o_c': 1,
                                          'amp_c': 1, 'h_c': 1, 'ppi_c': 1,
                                          tRNA: -1}

In [ ]:
for subreaction in me.subreaction_data:
    if type(subreaction.enzyme) == list:
        for enzyme in subreaction.enzyme:
            enzyme_met = Complex(enzyme)
            me.add_metabolites([enzyme_met])
    else:
        enzyme_met = Complex(subreaction.enzyme)
        me.add_metabolites([enzyme_met])        

In [ ]:
gb_filename = join(ecoli_files_dir,'NC_000913.2.gb')                                                                                    
TU_df = pandas.read_csv(join(ecoli_files_dir,'TUs_from_ecocyc.txt'), delimiter="\t", index_col=0)


sigma_to_RNAP_dict = transcription.sigma_factor_complex_to_rna_polymerase_dict

building.build_reactions_from_genbank(me, gb_filename, TU_df, tRNA_modifications=get_tRNA_modification_targets(),
                                      verbose=False,
                                      translation_terminators=ribosome.translation_stop_dict,
                                      frameshift_dict=ribosome.frameshift_dict,
                                     methionine_cleaved=ribosome.methionine_cleaved,
                                     folding_dict=ribosome.folding_dict,
                                     sigma_to_RNAP_dict=sigma_to_RNAP_dict,
                                     tRNA_to_codon=ribosome.tRNA_to_codon)

In [ ]:
ribosome.add_ribosome(me)
transcription.add_RNA_polymerase_complexes(me)

Make a dummy reactions. Add a dummy protein in as well.

## Associate the tRNA synthetases

The tRNA charging reactions were automatically added when loading the genome from the genbank file. However, the charging reactions still need to be made aware of the tRNA synthetases which are responsible.

In [ ]:
with open(join(ecoli_files_dir, "amino_acid_tRNA_synthetase.json"), "rb") as infile:
    aa_synthetase_dict = json.load(infile)
for data in me.tRNA_data:
    data.synthetase = str(aa_synthetase_dict[data.amino_acid])

### Add in complex Formation with modifications

In [ ]:
# ME_complex_dict is a dict of {'complex_id': [{'bnum' : count}]}
rna_components = {"b3123"} # component id should have RNA_ instead of protein_
ME_complex_dict = get_complex_to_bnum_dict(rna_components)
building.add_complex_stoichiometry_data(me, ME_complex_dict)

In [ ]:
modification_dict = get_protein_modification_dict()
building.add_complex_modification_data(me, modification_dict)

In [ ]:
# two different reactiond can add a lipoate modification.
# We create a separate ModificationData for each one
lipo = me.modification_data.get_by_id("mod_lipo_c")
alt_lipo = ModificationData("mod_lipo_c_alt", me)
#alt_lipo.stoichiometry = lipo.stoichiometry

lipo.stoichiometry = {"lipoamp_c": -1, "amp_c": 1}
lipo.enzyme = 'EG11796-MONOMER'
lipo.keff = 65.

alt_lipo.stoichiometry = {'EG50003-MONOMER_mod_pan4p_mod_lipo':-1,
                          'EG50003-MONOMER_mod_pan4p':1,
                          'h_c':-1,}
alt_lipo.enzyme = 'EG11591-MONOMER'
alt_lipo.keff = 65.

for cplx_data in lipo.get_complex_data():
    alt_cplx_data = ComplexData(cplx_data.id + "alt", me)
    alt_cplx_data.complex_id = cplx_data.complex_id
    alt_cplx_data.stoichiometry = cplx_data.stoichiometry
    alt_cplx_data.translocation = cplx_data.translocation
    alt_cplx_data.chaperones = cplx_data.chaperones
    alt_cplx_data.modifications = cplx_data.modifications.copy()
    alt_cplx_data.modifications[alt_lipo.id] = \
        alt_cplx_data.modifications.pop(lipo.id)

In [ ]:
# chaperones          
bmocogdp_chaperones = {'TMAOREDUCTI-CPLX':'EG12195-MONOMER', #bmocogdp
                 'DIMESULFREDUCT-CPLX':'G6849-MONOMER', #bmocogdp
                 'NITRATREDUCTA-CPLX':'NARJ-MONOMER', #bmocogdp
                 'NITRATREDUCTZ-CPLX':'NARW-MONOMER', #bmocogdp
                 'NAP-CPLX':'NAPD-MONOMER', #bmocogdp
                 'NAPAB-CPLX_NAPC-MONOMER':'NAPD-MONOMER'} #bmocogdp
for chaperone in set(bmocogdp_chaperones.values()):
    new_mod = ModificationData('mod_bmocogdp_c_' + chaperone, me)
    new_mod.enzyme = chaperone
    new_mod.stoichiometry = {'bmocogdp_c': -1}

for cplx_data in me.modification_data.get_by_id('mod_bmocogdp_c').get_complex_data():
    cplx_id = cplx_data.id.split('_mod')[0]
    if cplx_id in bmocogdp_chaperones:
        cplx_data.modifications['mod_bmocogdp_c_' + bmocogdp_chaperones[cplx_id]] = \
            cplx_data.modifications.pop('mod_bmocogdp_c')

 - Read "The CO and CN− ligands to the active site Fe in [NiFe]-hydrogenase of Escherichia coli have different metabolic origins" for insight into the metabolic origin of the NiFeCoCN2_c group

###Build all complex formation reactions

In [ ]:
for cplx_data in me.complex_data:
    formation = cplx_data.formation
    if formation:
        formation.update()
    else:
        cplx_data.create_complex_formation()

In [ ]:
codons = pandas.read_csv(join(ecoli_files_dir, "codon_usage.csv"), index_col=0)

In [ ]:
seq = "ATG"
for codon, row in codons.iterrows():
    if row.amino_acid == "Stop":
        continue
    seq += codon * int(row.per_1000 // 3)  # want roughly 300 aa
# get the most used stop codon
seq += codons[codons.amino_acid == "Stop"].sort_values("per_1000").index[-1]

In [ ]:
building.add_dummy_reactions(me, seq, update=True)
# we can now set the unmodeled protein fraction
me.unmodeled_protein_fraction = .45

TODO:
 - filter so known erpA dependent ones use that instead

In [ ]:
# these guys can transfer assembled iron sulfur clusters to the various enzymes
fes_transfer = {"erpA": "CPLX0-7617", "iscA": "IscA_tetra", "sufA": "CPLX0-7824"}

suf_cplx = ComplexData("sufBC2DES_pathway_complex", me)
suf_cplx.stoichiometry = {"CPLX0-1341": 1, "CPLX0-246_CPLX0-1342_mod_pydx5p": 1}
suf_cplx.create_complex_formation()
    
isc_cplx = ComplexData("iscUS_cyaY_pathway_complex", me) # could add chaperones into here
isc_cplx.stoichiometry = {"IscU": 1, "IscS_mod_2:pydx5p": 1, "EG11653-MONOMER": 1}
isc_cplx.create_complex_formation()

generic_fes_transfer = GenericData("generic_fes_transfer", me, ['CPLX0-7617', 'CPLX0-7824', 'IscA_tetra'])
generic_fes_transfer.create_reactions()

In [ ]:
me.modification_data.mod_2fe2s_c.enzyme = generic_fes_transfer.id
me.modification_data.mod_2fe2s_c.keff = 65.
me.modification_data.mod_4fe4s_c.enzyme = generic_fes_transfer.id
me.modification_data.mod_4fe4s_c.keff = 65.

In [ ]:
# Add known specific chaperone
fes_chaperones = {'CPLX0-1762':'G6712-MONOMER'} # FE-S modification
for chaperone in set(fes_chaperones.values()):
    new_mod = ModificationData('mod_2fe2s_c_' + chaperone, me)
    new_mod.enzyme = chaperone
    new_mod.stoichiometry = {'2fe2s_c': -1}
for cplx_data in me.modification_data.get_by_id('mod_2fe2s_c').get_complex_data():
    cplx_id = cplx_data.id.split('_mod')[0]
    if cplx_id in fes_chaperones:
        cplx_data.modifications['mod_2fe2s_c_' + fes_chaperones[cplx_id]] = \
            cplx_data.modifications.pop('mod_2fe2s_c')

Crutch reactions for mets that are blocked. TODO remove

## Associate Complexes with Reactions

In [ ]:
# associate reaction id with the old ME complex id (including modifications)
rxnToModCplxDict = get_reaction_to_complex(generic=False)

In [ ]:
rxn_info = get_reaction_info_frame()
for reaction_data in me.stoichiometric_data:
    building.add_metabolic_reactions(me, reaction_data, rxnToModCplxDict, rxn_info, update=True,
                                     create_new=True)

Sometimes multiple entities can perform the same role. To prevent a combinatorial explosion of possibilities, we can create  "generic" version, where any of those entities can fill in.

In [ ]:
for generic, components in ecoli_k12.generic_dict.items():
    GenericData(generic, me, components).create_reactions()

Rebuild transcription and translation to use tRNA (now that tRNA synthetase complexes are in the model

In [ ]:
apoACP = me.metabolites.get_by_id('EG50003-MONOMER_mod_pan4p')
for reaction in me.metabolites.get_by_id('EG50003-MONOMER_mod_pan4p').reactions:
    if apoACP in reaction.reactants:
        reaction.complex_dilution_list = {apoACP.id}
        reaction.update()
me.reactions.acp_lipoate_synthase_FWD_None.complex_dilution_list = {'CPLX0-782_mod_2:4fe4s'}
me.reactions.MOADSUx1_FWD_CPLX_dummy.complex_dilution_list = {'EG11597-MONOMER_mod_amp'}

## Add in translocation

In [ ]:
translocPath = pandas.read_csv(join(ecoli_files_dir, "translocation_pathways.txt"),sep='\t')
for index, row in translocPath.iterrows():
    translocRxn = ProteinTranslocationData(row.Reaction_name, me)
    translocRxn.keff = float(row.Keff)
    translocRxn.costs_complexes = row.Complexes.split(' AND ')

In [ ]:
proteins_sa_coeff_inner={}

transloc = pandas.read_csv(join(ecoli_files_dir, "peptide_compartment_and_pathways2.txt"), sep='\t', comment="#")
for index, row in transloc.iterrows():
    me.metabolites.get_by_id(row.Complex).compartment = row.Complex_compartment
    me.metabolites.get_by_id('protein_'+row.Protein.split('(')[0]).compartment = row.Protein_compartment
    pathway = False
    if 's' in row.translocase_pathway:
        pathway = True
        me.translocation_pathways.srp_translocation.add_translocation_cost(me,row.Complex,row.Protein)
        
        ## This is in preparation for membrane constraint
        mass = me.translation_data.get_by_id(row.Protein.split('(')[0]).mass
        if row.Complex in proteins_sa_coeff_inner.keys():
            proteins_sa_coeff_inner[row.Complex]+=mass*1.21/42.*2
        else:
            proteins_sa_coeff_inner[row.Complex]=mass*1.21/42.*2
    if 'r' in row.translocase_pathway:
        pathway = True
        me.translocation_pathways.srp_translocation.add_translocation_cost(me,row.Complex,row.Protein)       
    if 'p' in row.translocase_pathway:
        pathway = True
        me.translocation_pathways.srp_yidC_translocation.add_translocation_cost(me,row.Complex,row.Protein)         
    if 't' in row.translocase_pathway:
        pathway = True
        me.translocation_pathways.tat_translocation.add_translocation_cost(me,row.Complex,row.Protein)
    if 'a' in row.translocase_pathway:
        pathway = True
        me.translocation_pathways.srp_translocation.add_translocation_cost(me,row.Complex,row.Protein)
        me.translocation_pathways.secA_translocation.add_translocation_cost(me,row.Complex,row.Protein)
    if 'l' in row.translocase_pathway:
        pathway = True
        me.translocation_pathways.lol_translocation.add_translocation_cost(me,row.Complex,row.Protein)
    if 'b' in row.translocase_pathway:
        pathway = True
        me.translocation_pathways.bam_translocation.add_translocation_cost(me,row.Complex,row.Protein)
    if 'y' in row.translocase_pathway:
        pathway = True
        me.translocation_pathways.yidC_translocation.add_translocation_cost(me,row.Complex,row.Protein)        
    if not pathway:
        print row.Complex, row.translocase_pathway
    #if row.translocase_pathway != 'n':
    #    print row.Complex, row.translocase_pathway        

## Make RNA splicing machinery

In [ ]:
excision_types = ['rRNA_containing', 'monocistronic',
                  'polycistronic_wout_rRNA']
for excision_type in excision_types:
    complex_data =  ComplexData(excision_type + "_excision_machinery", me)
    complex_data.stoichiometry = {i: 1 for i in ecoli_k12.excision_machinery[excision_type]}
    complex_data.create_complex_formation()
    modification = ModificationData(excision_type + "_excision", me)
    modification.stoichiometry = {'h2o_c': -1, 'h_c': 1}
    modification.enzyme = complex_data.id

for t in me.transcription_data:
    n_excised = sum(t.excised_bases.values())
    n_cuts = len(t.RNA_products) * 2
    if n_excised == 0 or n_cuts == 0:
        continue
    RNA_types = list(t.RNA_types)
    n_tRNA = RNA_types.count("tRNA")
    
    if "rRNA" in set(RNA_types):
        t.modifications["rRNA_containing_excision"] = n_cuts
    elif n_tRNA == 1:
        t.modifications["monocistronic_excision"] = n_cuts
    elif n_tRNA > 1:
        t.modifications["polycistronic_wout_rRNA_excision"] = n_cuts
    else:  # only applies to rnpB
        t.modifications["monocistronic_excision"] = n_cuts

### Set GAM and NGAM

In [ ]:
me.stoichiometric_data.ATPM.lower_bound = 1. #10.33 # NGAM from iML1515

In [ ]:
GAM = 40.
gam_components = {
    "atp_c": -1 * GAM,
    "h2o_c": -1 * GAM,
    "adp_c": 1 * GAM,
    "h_c": 1 * GAM,
    "pi_c": 1 * GAM,}
biomass_components = {
    "DNA_biomass": -1e-3,

    "glycogen_c": -.023 / (me.metabolites.glycogen_c.formula_weight / 1000.),
    # cell wall
    'murein5px4p_p': -0.01389,
    'kdo2lipid4_e': -0.01945,
    'pe160_c': -0.01786,
    'pe160_p': -0.04594,
    'pe161_c': -0.02105,
    'pe161_p': -0.05415,
    
}
component_mass = sum(me.metabolites.get_by_id(c).formula_weight / 1000. * -v
                     for c, v in biomass_components.items())
me.reactions.biomass_dilution.add_metabolites(gam_components)
me.reactions.biomass_dilution.add_metabolites(biomass_components)
me.reactions.biomass_dilution.add_metabolites({'biomass': component_mass})

## Adjust other reaction parameters

In [ ]:
me.reactions.dummy_reaction_FWD_CPLX_dummy.objective_coefficient = 1.
me.reactions.EX_glc__D_e.lower_bound = -1000
# Turn off reactions that throw off results as in iOL
KO_list = ['DHPTDNR','DHPTDNRN', 'SUCASPtpp','SUCFUMtpp', 'SUCMALtpp', 'SUCTARTtpp', 
           'CAT', 'FHL', 'SPODM', 'SPODMpp']
for reaction in KO_list:
    data = me.stoichiometric_data.get_by_id(reaction)
    data.lower_bound = 0
    data.upper_bound = 0

## Attempt to set keffs

In [ ]:
keffs = get_reaction_keffs(me, verbose=True)
for reaction_id, keff in keffs.items():
    me.reactions.get_by_id(reaction_id).keff = keff

## Clean up and update everything

In [ ]:
me.update()
with open("prototype_52_Full.pickle", "wb") as outfile:
    cPickle.dump(me, outfile)
me.prune()

with open("prototype_52.pickle", "wb") as outfile:
    cPickle.dump(me, outfile)
with open("prototype_52_expressions.pickle", "wb") as outfile:
    cloudpickle.dump(expressions, outfile)

In [ ]:
n_genes = len(me.metabolites.query(re.compile('RNA_b[0-9]')))
print("number of genes in the model %d (%.2f%%)" % (n_genes, n_genes * 100. / (1678)))

## Solve

Save for future recomputation

In [ ]:
# TODO: cu is needed for ndhII activity. this wasn't in the old ME supplement but the model had
# cu uptake. There is no cu_p->cu_c reaction
me.reactions.EX_cu_e.lower_bound = -1000  
rxn = cobra.Reaction('cu_source')
me.add_reaction(rxn)
rxn.reaction = 'cu_c <-'

for rxn in me.metabolites.get_by_id('NADH-DHII-MONOMER_mod_mg2_mod_cu_mod_fad').metabolic_reactions:
    rxn.stoichiometric_data.stoichiometry['ndh2_constraint'] = 1
    rxn.update(create_new=True)
for rxn in me.metabolites.get_by_id('NADH-DHI-CPLX_mod_2fe2s_mod_4fe4s_mod_fmn').metabolic_reactions:
    rxn.stoichiometric_data.stoichiometry['ndh1_constraint'] = 1
    rxn.update(create_new=True)
rxn = cobra.Reaction('ndh_flux_split_constraint')
me.add_reaction(rxn)
rxn.reaction = 'ndh1_constraint + ndh2_constraint ->'

In [ ]:
me.reactions.EX_pqq_e.lower_bound = 0
me.reactions.EX_pqq_e.upper_bound = 0

In [ ]:
biomass = me.metabolites.biomass
me.reactions.protein_biomass_dilution.add_metabolites({biomass:.45})
me.reactions.protein_biomass_dilution.reaction

In [ ]:
expressions = compile_expressions(me)

In [ ]:
binary_search(me, compiled_expressions=expressions, debug=True,
              min_mu = 0.9, max_mu = 1.1, mu_accuracy=1e-5)

In [ ]:
import escher
view = escher.Builder("iJO1366.Central metabolism")
view.reaction_data = me.get_metabolic_flux()
view.display_in_notebook()